***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-8898  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec  4 11:31 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '206013310_313191645_final_project' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text", "id").rdd # Remark!! when we want to index the title and anchor, add here!!

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
total_number_of_docs_N = parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [ ]:
#start addition-------------------------
import json
# Opening JSON file
test_set_dict = None
with open('queries_train.json') as json_file:
    test_set_dict = json.load(json_file)
test_set_terms = set(' '.join(' '.join(list(test_set_dict.keys())).split('?')).split())

def our_sum(lst):
  summ = 0
  for i in lst:
    summ+=i
  return summ
#end addition------------------------------

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

from functools import total_ordering

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  set_of_toks = set(tokens)
  len_before = len(set_of_toks)
  len_after = len(set_of_toks - test_set_terms)
  if len_after == len_before:
    return []
  tf_counter = Counter()
  for tok in tokens:
    if tok not in all_stopwords:
      tf_counter[tok] += 1
  to_ret = []
  for item in tf_counter.items():
    to_ret.append((item[0],(id,item[1])))
  return to_ret

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl,key=lambda x: x[0])

def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

#start addition-------------------------
def vector_size(docid_to_freq,df_dict): #(12, [('anarchism', 125), ('political', 26), ('philosophy', 6), ('movement', 23), ('sceptical', 1), ('authority', 18), ('rejects', 1), ('involuntary', 1), ('coercive', 2), ('forms', 9), ('hierarchy', 4), ('calls', 3), ('abolition', 3), ('state', 50), ('holds', 2), ('undesirable', 1), ('unnecessary', 1), ('harmful', 1), ('historically', 3), ('far-left', 2), ('usually', 4), ('described', 2), ('alongside', 5), ('libertarian', 14), ('marxism', 6), ('wing', 3), ('socialism', 15), ('socialist', 14), ('strong', 2), ('historical', 4), ('association', 5), ('anti-capitalism', 3), ('history', 5), ('anarchy', 9), ('goes', 1), ('back', 2), ('prehistory', 1), ('humans', 4), ('arguably', 1), ('lived', 1), ('anarchic', 2), ('societies', 2), ('long', 2), ('establishment', 3), ('formal', 3), ('states', 8), ('realms', 1), ('empires', 1), ('rise', 1), ('organised', 2), ('hierarchical', 3), ('bodies', 1), ('scepticism', 1), ('toward', 3), ('rose', 1), ('19th', 6), ('century', 20), ('self-conscious', 1), ('emerged', 3), ('latter', 2), ('half', 2), ('first', 13), ('decades', 3), ('20th', 8), ('anarchist', 89), ('flourished', 2), ('parts', 1), ('world', 13), ('significant', 8), ('role', 6), ('workers', 11), ('struggles', 1), ('emancipation', 1), ('various', 15), ('schools', 13), ('thought', 12), ('formed', 5), ('period', 3), ('anarchists', 70), ('taken', 4), ('part', 8), ('several', 3), ('revolutions', 2), ('notably', 2), ('spanish', 4), ('civil', 5), ('war', 11), ('whose', 6), ('end', 5), ('marked', 1), ('classical', 14), ('era', 12), ('last', 2), ('21st', 3), ('resurgent', 1), ('employs', 1), ('diversity', 2), ('tactics', 15), ('order', 3), ('meet', 1), ('ideal', 8), ('ends', 4), ('broadly', 1), ('separated', 1), ('revolutionary', 10), ('evolutionary', 5), ('overlap', 2), ('two', 7), ('merely', 2), ('descriptive', 1), ('aim', 4), ('bring', 2), ('violent', 6), ('turn', 5), ('past', 2), ('prefigure', 1), ('society', 23), ('would', 14), ('like', 1), ('criticism', 10), ('praxis', 2), ('played', 1), ('diverse', 4), ('areas', 2), ('human', 9), ('criticisms', 1), ('include', 6), ('claims', 4), ('internally', 1), ('inconsistent', 1), ('utopian', 3), ('etymology', 2), ('terminology', 1), ('definition', 4), ('wilhelm', 2), ('weitling', 2), ('example', 3), ('writer', 1), ('added', 1), ('theory', 10), ('without', 10), ('using', 1), ('exact', 1), ('term', 4), ('etymological', 1), ('origin', 1), ('ancient', 2), ('greek', 1), ('anarkhia', 1), ('meaning', 3), ('ruler', 3), ('composed', 2), ('prefix', 1), ('word', 1), ('arkhos', 1), ('leader', 2), ('suffix', 1), ('ism', 1), ('denotes', 1), ('ideological', 2), ('current', 6), ('favours', 2), ('appears', 1), ('english', 2), ('1642', 1), ('anarchisme', 1), ('1539', 1), ('early', 6), ('usages', 1), ('emphasised', 1), ('sense', 3), ('disorder', 1), ('factions', 2), ('within', 12), ('french', 6), ('revolution', 10), ('labelled', 1), ('opponents', 1), ('although', 7), ('accused', 1), ('shared', 2), ('many', 16), ('views', 3), ('later', 6), ('revolutionaries', 2), ('william', 6), ('godwin', 4), ('1756', 1), ('1836', 1), ('1808', 1), ('1871', 2), ('contribute', 1), ('doctrines', 2), ('next', 1), ('generation', 1), ('use', 7), ('describing', 1), ('beliefs', 2), ('philosopher', 3), ('call', 2), ('pierre-joseph', 5), ('proudhon', 6), ('1809', 1), ('1865', 1), ('marking', 1), ('birth', 2), ('mid-19th', 1), ('since', 4), ('1890s', 2), ('beginning', 2), ('france', 3), ('libertarianism', 4), ('often', 6), ('used', 5), ('synonym', 3), ('still', 5), ('common', 8), ('outside', 1), ('united', 4), ('hand', 4), ('refer', 2), ('individualistic', 1), ('free-market', 2), ('referring', 2), ('largely', 6), ('synonymous', 1), ('recently', 1), ('diluted', 1), ('wider', 3), ('adoption', 1), ('ideologically', 3), ('disparate', 2), ('groups', 14), ('including', 3), ('new', 7), ('left', 7), ('marxists', 4), ('associate', 1), ('authoritarian', 2), ('socialists', 3), ('vanguard', 1), ('party', 2), ('well', 4), ('extreme', 2), ('liber
  freq_list = docid_to_freq[2] # list of tuples
  lst_tf_idf = []
  doc_size = docid_to_freq[1]
  for freq in freq_list:
    term = freq[0]
    num = freq[1]
    df = df_dict[term]
    lst_tf_idf.append(((num/doc_size) * math.log(total_number_of_docs_N/df ,2))**2)
  return (docid_to_freq[0], our_sum(lst_tf_idf)**0.5)
#end addition------------------------------

def partition_postings_and_write(postings):
  temp = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1])))
  temp = temp.groupByKey()
  return temp.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))

In [ ]:
#start addition-------------------------
doc_tf_vector_not_filtered = doc_text_pairs.map(lambda x: (x[1], word_count(x[0], x[1])))

doc_tf_vector = doc_tf_vector_not_filtered.filter(lambda x: len(x[1]) > 0)

doc_len = doc_tf_vector.map(lambda x: (x[0], our_sum([i[1][1] for i in x[1]]))) ## Di # 2 # key = dicId!!! value = doc len

doc_tf_unnormalized = doc_tf_vector.map(lambda x:(x[0], [(i[0],i[1][1]) for i in x[1]])) ## f_(i,j) # 3 # key = docId!!! value = tf vector

word_counts_not_filtered = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

word_counts = word_counts_not_filtered.filter(lambda x: len(x[1]) > 0)

postings = word_counts.groupByKey().mapValues(reduce_word_counts)

postings_filtered = postings.filter(lambda x: len(x[1])>10)

df_for_terms = postings_filtered.map(lambda x: (x[0], len(x[1])))

term_df = df_for_terms.collectAsMap()

doc_tf_unnormalized_with_doc_len = doc_tf_unnormalized.map(lambda x : (x[0],our_sum([i[1] for i in x[1]]),x[1]))

doc_vector_size = doc_tf_unnormalized_with_doc_len.map(lambda x : vector_size(x,term_df))
#end addition------------------------------
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
body_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
body_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
body_index.df = term_df
body_index.doc_len = doc_len.collectAsMap()
body_index.doc_vector_size = doc_vector_size.collectAsMap()
body_index.N = len(body_index.doc_len)
# write the global stats out
body_index.write_index('.', 'body_index_for_only_queries_docs_GCP')
# upload to gs
index_src = "body_index_for_only_queries_docs_GCP.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2021-12-04T13:30:16Z  gs://206013310assignment3/postings_gcp/index.pkl
TOTAL: 1 objects, 19343778 bytes (18.45 MiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  edges = pages.flatMapValues(lambda x: x).map(lambda x: ((x[0],x[1][0]),None)).combineByKey(lambda x: x, lambda x, _: x, lambda x, _: x).map(lambda x: x[0])
  vertices = edges.flatMap(lambda x: x).distinct().map(lambda x: (x,))
  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160779|
|  10568|  5385.34926364204|
|  32927| 5282.081575765276|
|  30680|  5128.23370960412|
|5843419|  4957.56768626387|
|  68253|  4769.27826535516|
|  31717|  4486.35018054831|
|  11867|  4146.41465091277|
|  14533| 3996.466440885503|
| 645042| 3531.627089803743|
|  17867|3246.0983906041406|
|5042916| 2991.945739166178|
|4689264|2982.3248830417474|
|  14532|2934.7468292031717|
|  25391|2903.5462235133973|
|   5405|2891.4163291546356|
|4764461|2834.3669873326608|
|  15573| 2783.865118158838|
|   9316| 2782.039646413769|
|8569916|2775.2861918400154|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://206013310assignment3/postings_gcp


In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 3.67 
print(f'I used {cost} USD credit during the course of this assignment')

I used 0 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag

AssertionError: 